In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('../Resources/train.csv')
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [13]:
df["Date"] = pd.to_datetime(df["Date"])
df["Date"] = pd.to_numeric(df["Date"])

In [14]:
from sklearn.model_selection import train_test_split
y = df["IsHoliday"]
X = df.drop(columns=["IsHoliday"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [15]:
X_train.head()

,Store,Dept,Date,Weekly_Sales
288095,30,11,1346371200000000000,616.37
307561,32,28,1278633600000000000,323.94
266695,27,97,1288310400000000000,19290.44
161724,17,32,1278028800000000000,5856.51
55059,6,49,1306454400000000000,10184.33


In [16]:
X_train.dtypes

Store             int64
Dept              int64
Date              int64
Weekly_Sales    float64
dtype: object

In [ ]:
train["Date"] = pd.to_datetime(train["Date"])

In [17]:
#Pre-processing
#Scale the data using LabelEncoder and MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Shubha\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [18]:

import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

In [19]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [20]:
y_train_categorical.shape

(316177, 2)

In [29]:
# Train the Model
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [30]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_8 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 202       
Total params: 10,802
Trainable params: 10,802
Non-trainable params: 0
_________________________________________________________________


In [32]:
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/60
 - 20s - loss: 0.2428 - acc: 0.9289
Epoch 2/60


C:\Users\Shubha\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\keras\callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 19s - loss: 0.2211 - acc: 0.9278
Epoch 3/60
 - 18s - loss: 0.2091 - acc: 0.9314
Epoch 4/60
 - 18s - loss: 0.1956 - acc: 0.9364
Epoch 5/60
 - 18s - loss: 0.1831 - acc: 0.9367
Epoch 6/60
 - 18s - loss: 0.1761 - acc: 0.9379
Epoch 7/60
 - 18s - loss: 0.1706 - acc: 0.9390
Epoch 8/60
 - 18s - loss: 0.1648 - acc: 0.9402
Epoch 9/60
 - 18s - loss: 0.1595 - acc: 0.9414
Epoch 10/60
 - 18s - loss: 0.1588 - acc: 0.9419
Epoch 11/60
 - 18s - loss: 0.2347 - acc: 0.9376
Epoch 12/60
 - 18s - loss: 0.2610 - acc: 0.9361
Epoch 13/60
 - 18s - loss: 0.2581 - acc: 0.9366
Epoch 14/60
 - 18s - loss: 0.2517 - acc: 0.9380
Epoch 15/60
 - 18s - loss: 0.2589 - acc: 0.9394
Epoch 16/60
 - 18s - loss: 0.3647 - acc: 0.9311
Epoch 17/60
 - 18s - loss: 0.3691 - acc: 0.9306
Epoch 18/60
 - 18s - loss: 0.3712 - acc: 0.9309
Epoch 19/60
 - 18s - loss: 0.3725 - acc: 0.9297
Epoch 20/60
 - 18s - loss: 0.3677 - acc: 0.9305
Epoch 21/60
 - 18s - loss: 0.3609 - acc: 0.9332
Epoch 22/60
 - 18s - loss: 0.3582 - acc: 0.9330
Epoch 23/60

In [33]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.3854605958014736, Accuracy: 0.9716869241795907


In [43]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [44]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {y_test}")

Predicted classes: [False False False ... False False False]
Actual Labels: 151791    False
320215    False
19956     False
127545    False
129732    False
180707    False
42161     False
219432    False
137145    False
109125    False
398144    False
384502    False
243414    False
133706    False
179719    False
175260    False
322386     True
400475    False
343836    False
102243    False
221179    False
290243    False
253183    False
118482    False
34563     False
105358    False
398656    False
110673    False
384872     True
7089      False
          ...  
73618     False
310844    False
209979    False
334870    False
106880    False
83216     False
36538      True
140426    False
52712     False
369783    False
245221    False
166334    False
42631     False
121329     True
171387    False
367364    False
347175     True
231744    False
96797     False
276957    False
181918    False
231223    False
179274    False
6665      False
359475    False
268681    False
402598    Fa

In [62]:
y_test["IsHoliday"]

KeyError: 'IsHoliday'